## Programming Assignment 3

Download the following text file (right click and select "Save As..."): kargerMinCut.txt

The file contains the adjacency list representation of a simple undirected graph. There are 200 vertices labeled 1 to 200. The first column in the file represents the vertex label, and the particular row (other entries except the first column) tells all the vertices that the vertex is adjacent to. So for example, the 6th row looks like : "6	155	56	52	120 ......". This just means that the vertex with label 6 is adjacent to (i.e., shares an edge with) the vertices with labels 155,56,52,120,......,etc

Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut. (HINT: Note that you'll have to figure out an implementation of edge contractions. Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction. But you should also think about more efficient implementations.) (WARNING: As per the video lectures, please make sure to run the algorithm many times with different random seeds, and remember the smallest cut that you ever find.) Write your numeric answer in the space provided. So e.g., if your answer is 5, just type 5 in the space provided.

## 要点回顾：
+ 时间复杂度 $\Omega(n^2m)$.
+ 重复试验的次数超过 $n^2\ln n$.
+ 

参考资料：https://blog.csdn.net/yuli_dai/article/details/9720933

In [1]:
import random
import copy
import time
 
def contract(ver, e): 
    while len(ver) > 2: #create a new graph every time (not efficient)
        ind = random.randrange(0, len(e))      
        [u, v] = e.pop(ind)   #pick a edge randomly
        ver.remove(v)    #remove v from vertices
        newEdge = list()
        for i in range(len(e)):
            if e[i][0] == v: e[i][0] = u
            elif e[i][1] == v: e[i][1] = u
            if e[i][0] != e[i][1]: newEdge.append(e[i])   # remove self-loops
        e = newEdge
    return(len(e))  #return the number of the remained edges 
 
if __name__ == '__main__':
    f = open('kargerMinCut.txt')
    _f = list(f) 
    edges = list()          #initialize vertices and edges
    vertices = list()
    for i in range(len(_f)):     #got 2517 different edges
        s = _f[i].split()
        vertices.append(int(s[0]))
        for j in range(1, len(s)):
            if [int(s[j]), int(s[0])] not in edges:
                edges.append([int(s[0]), int(s[j])])  
 
    result = list()
    starttime = time.clock() 
    for i in range(2000):  #we take n^2logn times so that the Pr(allfail) <= 1/n where n is the number of vertics
        v = copy.deepcopy(vertices)   #notice: deepcopy
        e = copy.deepcopy(edges)
        r = contract(v, e)
        result.append(r)
    endtime = time.clock()
    #print(result)
    print(min(result))
    print(endtime - starttime)

C:\Users\89487\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


17
449.6039704519999


C:\Users\89487\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


先举课堂上给的例子：

In [50]:
# 注意这里是我们图的存储方式：G 是图的字典表示，键是顶点，值是列表表示所有与该顶点相连接的顶点
G = {1:[2,3], 2:[1,3,4], 3:[1,2,4], 4:[2,3]}

第一种合并方式：

In [94]:
merge(G, 1, 3)

{1: [2, 2, 4], 2: [1, 1, 4], 4: [2, 1]}

In [95]:
merge(G, 1, 2)

{1: [4, 4], 4: [1, 1]}

最后的剩下的确实是最小割边。

In [119]:
def merge(G, u, v):
    """ 给定图 G 以及 G 中的两个顶点 u, v，
    返回合并 u, v 后的新图 G. 输入的 G 是图的字典表示，键是顶点，值是列表表示所有与该顶点相连接的顶点。"""
    G[u] = G[u] + G[v]
    del G[v]
        
    for ver in G:
        # 把所有 G 的 v 点标为 u
        G[ver] = [u if x == v else x for x in G[ver]]
    
    # 删除自环
    while u in G[u]:
        G[u].remove(u)
    return G

第二种合并方式：

In [51]:
merge(G, 1, 3)

{1: [2, 2, 4], 2: [1, 1, 4], 4: [2, 1]}

In [52]:
merge(G, 2, 4)

{1: [2, 2, 2], 2: [1, 1, 1]}

最后的剩下的就不是最小割边。

## 最终代码

In [7]:
import random

def merge(G, u, v):
    """ 给定图 G 以及 G 中的两个顶点 u, v，
    返回合并 u, v 后的新图 G. 输入的 G 是图的字典表示，键是顶点，值是列表表示所有与该顶点相连接的顶点。"""
    G[u] = G[u] + G[v]
    del G[v]
        
    for ver in G:
        # 把所有 G 的 v 点标为 u
        G[ver] = [u if x == v else x for x in G[ver]]
    
    # 删除自环
    while u in G[u]:
        G[u].remove(u)
    return G

def kargerMinCut(G):
    """ 任给一个图 G，返回最终利用 Random Contraction Algorithm 得到的图 G."""
    while len(G) > 2:
        v = random.choice(list(G.keys()))
        u = random.choice(G[v])

        merge(G, u, v)
    return G

Programming Assignment 3 的答案

In [44]:
f = open('kargerMinCut.txt')
GG = {}
_f = list(f) 
for i in range(len(_f)):
    s = _f[i].split()
    GG[s[0]] = s[1:]

In [45]:
import copy

A = []
for i in range(2000):
    G = GG.copy()
    kargerMinCut(G)
    A.append(len(list(G.values())[0]))
print(min(A))    

17
